In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# Opening JSON file
f = open('data.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

In [3]:
print(type(data))

<class 'dict'>


In [4]:
%load_ext autoreload
%autoreload 2
import pipeline_entity_quote as pipeline

/Users/oooyyyzzz/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /Users/oooyyyzzz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Result_dict

num_entity: number of entities;     
num_entity_type: number of entity types;
num_org: number of ORG(organization entities);        


# Trial1: get number of entities

In [5]:
result_dict = {}

In [5]:
for uid, info in data.items():
    text = info['text']
    doc, token_lst = pipeline.tokenize_text(text)
    entity_dict, num_entity = pipeline.detect_entity(doc)
    num_entity_type = len(entity_dict)
    
    break

9
{'ORDINAL': ['first'], 'NORP': ['Democrat', 'Coloradans', 'Republicans', 'Democrats', 'Democratic', 'Republican'], 'MONEY': ['21 trillion dollars'], 'GPE': ['Puerto Rico', 'California', 'Illinois', 'New York', 'Colorado'], 'PERSON': ['Jared Polis', 'Dianne Primavera', 'Michele Poague', 'Patriot Boot Camp', 'Walker Stapleton', 'Eric Bodenstab', 'Scott Helker', 'Polis', 'Stapleton', 'Bill Ritter', 'Bill Hammons'], 'ORG': ['Unity Party of America', 'the Unity Party', 'General Election', 'the Federal Government', 'the State Assembly', 'Medicaid', 'Unity Party', 'Rank', 'Senate', 'Medicare'], 'DATE': ['the next decade', 'Just two years ago', 'November', '100-day', 'Amendment 69', 'the day'], 'CARDINAL': ['two', '49', 'hundreds', '1,000', 'four', '3', '2', '1', 'hundreds of thousands', 'three'], 'PERCENT': ['the other 90 percent', 'only 10 percent', '100 percent', 'nearly 80 percent']}
54


# Trial2: get quotation sentiment

In [10]:
import en_core_web_sm

from textacy import extract as ex
nlp = en_core_web_sm.load()
doc=nlp(" \"Kurds are pitted against Kurds , Iraqis against Kurds , Turks against Kurds -- and the American planes just circle overhead without intervening , \" the minister said")
a=ex.direct_quotations(doc)
for item in a: print (item)

    
# output type reference: 
# https://textacy.readthedocs.io/en/latest/api_reference/extract.html?highlight=extract#textacy.extract.triples.DQTriple 


DQTriple(speaker=[minister], cue=[said], content="Kurds are pitted against Kurds , Iraqis against Kurds , Turks against Kurds -- and the American planes just circle overhead without intervening , ")


In [13]:
test_text = data["0526bd0d-41bb-43fe-bdc9-dabeea9dcfaa"]["text"]
test_text

'MANILA - The Senate on Monday passed on third reading the Alternative Learning System, which allows out-of-school youth, persons with disabilities, indigenous peoples, and other underserved communities to receive education "tailored to respond to their learning needs and life circumstances."\n\nTwenty-two senators voted in favor of Senate Bill No. 1365, which sought to "guarantee equitable opportunity for all learners."\n\n"The ALS Act is, in its very essence, a bill about second chances," said principal author Sen. Sherwin Gatchalian in a statement.\n\n"It is a bill about providing opportunities for a better life to our fellow Filipinos who have fallen into hard times,” said the senator, who chairs the Senate Committee on Basic Education Arts and Culture.\n\nUnder the measure, all cities and municipalities will be required to have at least 1 ALS center that would provide "a mix of learning modalities" such as "digital learning, modular instruction, and radio and television-based inst

In [51]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from statistics import mean

def get_sentiment(text):
    '''
    subjectivity: 0.0 = objective, 1.0 = subjective
    polarity: measures level of approval/disapproval
    '''
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    doc = nlp(text)
    subjectivity = doc._.subjectivity
    polarity = doc._.polarity
    return subjectivity, polarity

In [70]:
def get_quotation_sentiment(text):
    '''
    Get the number of direct quotations,
    average subjectivity score and polarity score of quotations, None if no direct quotations
    '''
    quotes = pipeline.get_quotes(text)
    sub_score_lst = []
    pol_score_lst = []
    num_quote = 0
    for item in quotes:
        item_text = str(item.content) #get quotation text
#         print(item_text)
        subjectivity, polarity = get_sentiment(item_text)
        sub_score_lst.append(subjectivity)
        pol_score_lst.append(polarity)
        num_quote += 1
    if num_quote == 0:
        return num_quote, None, None
    else:
        return num_quote, mean(sub_score_lst), mean(pol_score_lst)

In [64]:
get_quotation_sentiment(test_text)

num_quote: 1
num_quote: 2
num_quote: 3
num_quote: 4
num_quote: 5
num_quote: 6
num_quote: 7
num_quote: 8


In [71]:
for uid, info in data.items():
    text = info['text']
    num_quote, quote_subjectivity, quote_polarity = get_quotation_sentiment(text)
    break

# Pipeline


In [ ]:
result_dict = pipeline.get_training_features()

[nltk_data] Downloading package punkt to /Users/oooyyyzzz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
result_dict

In [ ]:
result_df = pd.DataFrame.from_dict(result_dict)
result_df

In [8]:
result_df = pd.DataFrame.from_dict(result_dict)
result_df

,204f5d99-c9de-4bce-ae2f-5e98085b4930,0526bd0d-41bb-43fe-bdc9-dabeea9dcfaa,33e5aa3f-af33-461d-9e51-16a552e6b985,d10f9f69-204a-422d-886d-ff59f787fe22,88ffc85f-b495-4f59-af53-08a92030f9b7,6c0221c9-f3c9-4f6b-800e-685e75b317c1,e10be83f-75b7-40f8-b041-5bbb9f3f6d77,d65803ad-3800-4a16-b02b-fef589314fab,c0823b0b-16ee-46e3-955e-1bfd50300c3e,7cc09e87-92e3-48c6-aa65-f96fe480f2b0,648f5cc1-57dd-4d8c-b469-c7a483b757c0
entity_num,54.0,22.000000,39.000000,33.0,36.0,15.000000,44.000000,122.000000,40.0,59.000000,23.000000
entity_type_num,9.0,8.000000,7.000000,8.0,8.0,7.000000,7.000000,12.000000,9.0,7.000000,7.000000
num_quote,0.0,8.000000,16.000000,NaN,NaN,3.000000,5.000000,12.000000,NaN,7.000000,4.000000
quote_subjectivity,NaN,0.128299,0.399621,NaN,NaN,0.022222,0.096667,0.377285,NaN,0.292354,0.490215
quote_polarity,NaN,0.048438,-0.031676,NaN,NaN,0.011111,0.045000,-0.045109,NaN,0.022103,-0.104672


In [ ]:
result_df.transpose()

In [ ]:
# small trial
def get_training_features():
    '''
    Pipeline of extracting entity and quotation features from dataset
    '''
    data = get_json_data('data.json')
    result_dict = {}
    count = 0 #
    for uid, info in data.items():
        
        count += 1
        text = info['text']
        doc, token_lst = tokenize_text(text)
        entity_dict, entity_num, entity_type_num = detect_entity(doc)
        try:
            num_quote, quote_subjectivity, quote_polarity = get_quotation_sentiment(text)
        except:
            num_quote, quote_subjectivity, quote_polarity = None, None, None
        result_dict[uid] = {}
        result_dict[uid]['entity_dict'] = entity_dict
        result_dict[uid]['entity_num'] = entity_num
        result_dict[uid]['entity_type_num'] = entity_type_num
        result_dict[uid]['num_quote'] = num_quote
        result_dict[uid]['quote_subjectivity'] = quote_subjectivity
        result_dict[uid]['quote_polarity'] = quote_polarity
        
        if count > 10:
            break
    return result_dict

In [ ]:
result_dict = pipeline.get_training_features()
result_dict